# Batch Scoring on IBM Cloud Pak for Data (ICP4D)

We are going to use this notebook to create and/or run a batch scoring job against a model that has previously been created and deployed to the Watson Machine Learning (WML) instance on Cloud Pak for Data (CP4D).

## 1.0 Install required packages


There are a couple of Python packages we will use in this notebook. We will install/upgrade the Watson Machine Learning (WML) client library to interact with the Watson Machine Learning service from this notebook. These package may already be installed by default on CP4D.

- WML Client: http://ibm-wml-api-pyclient.mybluemix.net/

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install --user ibm-watson-machine-learning==1.0.53 --upgrade | tail -n 1

In [3]:
import json
from ibm_watson_machine_learning import APIClient

## 2.0 Create Batch Deployment Job

### 2.1 Instantiate Watson Machine Learning Client

To interact with the local Watson Machine Learning instance, we will be using the Python SDK. 

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

<font color='red'>Replace the `username` and `password` values of `************` with your Cloud Pak for Data `username` and `password`. The value for `url` should match the `url` for your Cloud Pak for Data cluster, which you can get from the browser address bar (be sure to include the 'https://'.</font> The credentials should look something like this (these are example values, not the ones you will use):

`
wml_credentials = {
    "url": "https://zen.clusterid.us-south.containers.appdomain.cloud/",
    "username": "cp4duser",
    "password" : "cp4dpass",
    "instance_id": "openshift",
    "version" : "3.5"
}
`
#### NOTE: Make sure that there is no trailing forward slash `/` in the `url`

In [4]:
# Be sure to update these credentials before running the cell.
wml_credentials = {
                   "url": "https://zen-cpd-zen.**********.us-south.containers.appdomain.cloud/",
                   "username": "*************",
                   "password" : "*********",
                   "instance_id": "wml_local",
                   "version" : "3.5"
                  }

wml_client = APIClient(wml_credentials)

In [5]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -------------------------  ------------------------
ID                                    NAME                       CREATED
b90bbdc7-c2e9-4c36-9e1f-d41d181b41ef  CreditRiskDeploymentSpace  2021-03-08T03:51:46.591Z
------------------------------------  -------------------------  ------------------------


### 2.2 Find Deployment Space

We will try to find the `ID` for the deployment space you want to use and set it as the default space for the client.

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

- Update with the name of the deployment space where you have created the batch deployment.

In [6]:
# Be sure to update the name of the space with the one you want to use.
DEPLOYMENT_SPACE_NAME = 'CreditRiskDeploymentSpace'

In [7]:
all_spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in all_spaces:
    if space['entity']['name'] == DEPLOYMENT_SPACE_NAME:
        space_id = space["metadata"]["id"]
        print("\nDeployment Space ID: ", space_id)

if space_id is None:
    print("WARNING: Your space does not exist. Create a deployment space before proceeding.")
    # We could programmatically create the space.
    #space_id = wml_client.spaces.store(meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]


Deployment Space ID:  b90bbdc7-c2e9-4c36-9e1f-d41d181b41ef


In [8]:
# Now set the default space to the ID for your deployment space. If this is successful, you will see a 'SUCCESS' message.
wml_client.set.default_space(space_id)

'SUCCESS'

In [9]:
# These are the models and deployments we currently have in our deployment space.
wml_client.repository.list_models()
wml_client.deployments.list()

------------------------------------  ------------  ------------------------  ---------
ID                                    NAME          CREATED                   TYPE
e0851212-a6c1-4451-ba10-cfa975fcc121  JRTModel35v1  2021-03-12T17:31:00.002Z  mllib_2.4
------------------------------------  ------------  ------------------------  ---------
------------------------------------  -------------------------  -----  ------------------------
GUID                                  NAME                       STATE  CREATED
ffd90955-671e-4788-b687-e90b87fed472  SparkModelBatchDeployment  ready  2021-03-23T01:58:19.422Z
88b35fe7-6ad1-40e4-b8b0-713ca992b4fd  SparkChurnModelOnline      ready  2021-03-23T01:36:54.325Z
------------------------------------  -------------------------  -----  ------------------------


### 2.3 Find Batch Deployment

We will try to find the batch deployment which was created.

<font color=red>**<< UPDATE THE VARIABLES BELOW >>**</font>

- Update the variable with the name of the batch deployment you created previously.

In [10]:
DEPLOYMENT_NAME = 'SparkModelBatchDeployment'

In [12]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
deployment_details = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['id']
        deployment_details = deployment
        #print(json.dumps(deployment_details, indent=3))
        break

print("Deployment id: {}".format(deployment_uid))
wml_client.deployments.get_details(deployment_uid)

Deployment id: ffd90955-671e-4788-b687-e90b87fed472


{'entity': {'asset': {'id': 'e0851212-a6c1-4451-ba10-cfa975fcc121'},
  'batch': {},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'f3ebac7d-0a75-410c-8b48-a931428cc4c5',
   'name': 'XS',
   'num_nodes': 1},
  'name': 'SparkModelBatchDeployment',
  'space_id': 'b90bbdc7-c2e9-4c36-9e1f-d41d181b41ef',
  'status': {'state': 'ready'}},
 'metadata': {'created_at': '2021-03-23T01:58:19.422Z',
  'id': 'ffd90955-671e-4788-b687-e90b87fed472',
  'modified_at': '2021-03-23T01:58:19.422Z',
  'name': 'SparkModelBatchDeployment',
  'owner': '1000331005',
  'space_id': 'b90bbdc7-c2e9-4c36-9e1f-d41d181b41ef'}}

### 2.4 Get Batch Test Data

We will load some data to run the batch predictions.

In [13]:
import pandas as pd

from project_lib import Project
project = Project.access()

batch_set = pd.read_csv(project.get_file('German-Credit-Risk-SmallBatchSet.csv'))
batch_set = batch_set.drop('CustomerID', axis=1,errors = 'ignore')
batch_set.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,CurrentResidenceDuration,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker
0,0_to_200,4,all_credits_paid_back,car_new,250,100_to_500,less_1,2,female,none,3,real_estate,26,bank,rent,1,unskilled,1,none,yes
1,0_to_200,14,credits_paid_to_date,car_new,3148,less_100,1_to_4,3,male,none,3,car_other,41,none,own,2,skilled,1,none,yes
2,greater_200,19,credits_paid_to_date,radio_tv,5351,100_to_500,greater_7,4,male,none,3,savings_insurance,49,none,own,2,skilled,1,yes,yes
3,greater_200,34,outstanding_credit,other,5790,500_to_1000,greater_7,5,male,none,4,car_other,44,stores,own,2,unskilled,1,yes,yes
4,less_0,4,all_credits_paid_back,car_new,250,100_to_500,less_1,1,female,none,1,real_estate,21,bank,rent,1,unskilled,1,none,yes


### 2.5 Create Job

We can now use the information about the deployment and the test data to create a new job against our batch deployment. We submit the data as inline payload and want the results (i.e predictions) stored in a CSV file.

In [14]:
import time
timestr = time.strftime("%Y%m%d_%H%M%S")
job_payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': batch_set.columns.values.tolist(),
        'values': batch_set.values.tolist()
    }],
    wml_client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
            "type": "data_asset",
            "connection": {},
            "location": {
                "name": "batchres_{}_{}.csv".format(timestr,deployment_uid),
                "description": "results"
            }
    }
}

job = wml_client.deployments.create_job(deployment_id=deployment_uid, meta_props=job_payload)
job_uid = wml_client.deployments.get_job_uid(job)

print('Job uid = {}'.format(job_uid))

Job uid = 157243ca-33c7-429f-bc32-b07d4c88130c


In [15]:
wml_client.deployments.list_jobs()

------------------------------------  ------  ------------------------  ------------------------------------
JOB-UID                               STATE   CREATED                   DEPLOYMENT-ID
157243ca-33c7-429f-bc32-b07d4c88130c  queued  2021-03-23T02:18:55.129Z  ffd90955-671e-4788-b687-e90b87fed472
------------------------------------  ------  ------------------------  ------------------------------------


## 3.0 Monitor Batch Job Status

The batch job is an async operation. We can use the identifier to track its progress. Below we will just poll until the job completes (or fails).

In [16]:
def poll_async_job(client, job_uid):
    import time
    while True:
        job_status = client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status['state']
        if state == 'completed' or 'fail' in state:
            return client.deployments.get_job_details(job_uid)
        time.sleep(5)
            
job_details = poll_async_job(wml_client, job_uid)

{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at

In [17]:
wml_client.deployments.list_jobs()

------------------------------------  ---------  ------------------------  ------------------------------------
JOB-UID                               STATE      CREATED                   DEPLOYMENT-ID
157243ca-33c7-429f-bc32-b07d4c88130c  completed  2021-03-23T02:18:55.129Z  ffd90955-671e-4788-b687-e90b87fed472
------------------------------------  ---------  ------------------------  ------------------------------------


### 3.1 Check Results

With the job complete, we can see the predictions. 

In [18]:
wml_client.deployments.get_job_details()

{'resources': [{'entity': {'deployment': {'id': 'ffd90955-671e-4788-b687-e90b87fed472'},
    'platform_job': {'job_id': '6a443a9e-ebba-4c59-914d-0ef0864af58c',
     'run_id': '7f45eef6-fa7f-4f99-8d61-5be2e2791dc8'},
    'scoring': {'input_data': [{'fields': ['CheckingStatus',
        'LoanDuration',
        'CreditHistory',
        'LoanPurpose',
        'LoanAmount',
        'ExistingSavings',
        'EmploymentDuration',
        'InstallmentPercent',
        'Sex',
        'OthersOnLoan',
        'CurrentResidenceDuration',
        'OwnsProperty',
        'Age',
        'InstallmentPlans',
        'Housing',
        'ExistingCreditsCount',
        'Job',
        'Dependents',
        'Telephone',
        'ForeignWorker'],
       'values': [['0_to_200',
         4,
         'all_credits_paid_back',
         'car_new',
         250,
         '100_to_500',
         'less_1',
         2,
         'female',
         'none',
         3,
         'real_estate',
         26,
         'bank'

In [19]:
print(json.dumps(job_details, indent=2))

{
  "entity": {
    "deployment": {
      "id": "ffd90955-671e-4788-b687-e90b87fed472"
    },
    "platform_job": {
      "job_id": "6a443a9e-ebba-4c59-914d-0ef0864af58c",
      "run_id": "7f45eef6-fa7f-4f99-8d61-5be2e2791dc8"
    },
    "scoring": {
      "input_data": [
        {
          "fields": [
            "CheckingStatus",
            "LoanDuration",
            "CreditHistory",
            "LoanPurpose",
            "LoanAmount",
            "ExistingSavings",
            "EmploymentDuration",
            "InstallmentPercent",
            "Sex",
            "OthersOnLoan",
            "CurrentResidenceDuration",
            "OwnsProperty",
            "Age",
            "InstallmentPlans",
            "Housing",
            "ExistingCreditsCount",
            "Job",
            "Dependents",
            "Telephone",
            "ForeignWorker"
          ],
          "values": [
            [
              "0_to_200",
              4,
              "all_credits_paid_back",
  

## Congratulations, you have created and submitted a job for batch scoring !